In [3]:
import json
import requests
import argparse
import pprint
import sys
import urllib
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('datasets_367098_716040_breweries_us.csv')
df.sample(25)

,brewery_name,type,address,website,state,state_breweries
1365,Rio Grande Brewing Co.,Brewpub,"3760 Hawkins St. NE, Albuquerque, New Mexico, ...",...,new-mexico,37
1433,Landmark Brewing Co.,ContractBrewery,"3650 James St., Rm 105, Syracuse, New York, 13206",http://www.landmarkbrewing.com/,new-york,107
649,"Orange Blossom Pilsner, LLC",ContractBrewery,"4410 Flagg St., Orlando, Florida, 32812",http://www.uniquebeers.com/obp/,florida,99
338,Confluence Brewing Co,Microbrewery,"1235 Thomas Beck Rd., Suite A, Des Moines, Iow...",http://confluencebrewing.com/,iowa,13
1914,T-Bonz Gill & Grill,Brewpub,"1028 Johnie Dodds Blvd., Mount Pleasant, South...",http://www.tbonz.com/,south-carolina,32
907,Circle V Brewing Co.,Brewpub-Closed,"8310 Craig St., Indianapolis, Indiana, 46250",...,indiana,31
2035,Healthy Brew,Microbrewery,"6435 Nine Mile Bridge Rd., Fort Worth, Texas, ...",http://www.healthybrew.com/,texas,90
451,Southern Sun Pub & Brewery,Brewpub,"627 South Broadway, Boulder, Colorado, 80305",http://www.mountainsunpub.com/,colorado,182
2129,Saint George Brewing Co.,Microbrewery,"204 Challanger Way, Hampton, Virginia, 23666",http://www.stgeorgebrewingco.com/,virginia,43
1379,Emdudo Station,Brewpub,"P.O. Box 154, Embudo, New Mexico, 87531",http://www.nets.com/webzarh/embudo.htm,new-mexico,37


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2407 entries, 0 to 2406
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   brewery_name     2407 non-null   object
 1   type             2407 non-null   object
 2   address          2407 non-null   object
 3   website          2399 non-null   object
 4   state            2407 non-null   object
 5   state_breweries  2407 non-null   int64 
dtypes: int64(1), object(5)
memory usage: 113.0+ KB


So I need to seprate and split the address to work with yelps API, also doesn't seem like a huge deal that they are a few missing websites 

In [6]:
new = df['address'].str.split(",", n= 4, expand = True)

In [7]:
df=df.join(new)

In [8]:
df.head()

,brewery_name,type,address,website,state,state_breweries,0,1,2,3,4
0,Valley Brewing Co.,Brewpub,"PO Box 4653, Stockton, California, 95204",http://www.valleybrew.com/,california,284,PO Box 4653,Stockton,California,95204,None
1,Valley Brewing Co.,Brewpub,"157 Adams St., Stockton, California, 95204",http://www.valleybrew.com/,california,284,157 Adams St.,Stockton,California,95204,None
2,Valley Brewing Co,Microbrewery,"1950 W Freemont, Stockton, California, 95203",http://www.valleybrew.com/,california,284,1950 W Freemont,Stockton,California,95203,None
3,Ukiah Brewing Company,Brewpub,"102 S. State St., Ukiah, California, 95482",http://www.ukiahbrewingco.com/,california,284,102 S. State St.,Ukiah,California,95482,None
4,Tustin Brewing Co.,Brewpub,"13011 Newport Ave. #100, Tustin, California, 9...",http://www.tustinbrewery.com/,california,284,13011 Newport Ave. #100,Tustin,California,92780,None


In [9]:
df=df.rename(columns={0:"address_2",1:"city",2:"state_2",3:"zipcode"})

In [10]:
df['state'].value_counts()

california        284
colorado          182
oregon            156
washington        151
new-york          107
pennsylvania      107
florida            99
michigan           94
texas              90
wisconsin          77
illinois           74
massachusetts      71
north-carolina     61
ohio               56
georgia            47
maine              44
maryland           43
virginia           43
new-jersey         41
minnesota          40
new-mexico         37
arizona            34
montana            32
south-carolina     32
indiana            31
tennessee          28
idaho              28
vermont            28
kansas             26
utah               21
missouri           21
new-hampshire      18
nevada             18
oklahoma           16
louisiana          15
nebraska           15
delaware           14
alaska             14
iowa               13
rhode-island       12
kentucky           12
south-dakota       11
connecticut        11
hawaii             10
washington-dc       9
wyoming   

In [11]:
df.state = df.state.replace('california','CA')
df.state = df.state.replace('colorado','CO')
df.state = df.state.replace('oregon', 'OR')
df.state =df.state.replace('washington','WA')
df.state = df.state.replace('new-york', 'NY')
df.state = df.state.replace('pennsylvania','PA')
df.state = df.state.replace('florida', 'FL')
df.state = df.state.replace('michigan','MI')
df.state = df.state.replace('texas','TX')
df.state = df.state.replace('wisconsin','WI')
df.state = df.state.replace('illinois','IL')
df.state = df.state.replace('massachusetts', 'MA')
df.state = df.state.replace('north-carolina','NC')
df.state = df.state.replace('ohio','OH')
df.state = df.state.replace('georgia', 'GA')
df.state = df.state.replace('maine','ME')
df.state = df.state.replace('virginia','VA')
df.state = df.state.replace('maryland','MD')
df.state = df.state.replace('new-jersey','NJ')
df.state = df.state.replace('minnesota', 'MN')
df.state = df.state.replace('new-mexico','NM')
df.state = df.state.replace('arizona','AZ')
df.state = df.state.replace('montana','MT')
df.state = df.state.replace('south-carolina','SC')
df.state = df.state.replace('indiana','IN')
df.state = df.state.replace('vermont','VT')
df.state = df.state.replace('tennessee', 'TN')
df.state = df.state.replace('idaho','ID')
df.state = df.state.replace('kansas','KS')
df.state = df.state.replace('missouri','MO')                        
df.state = df.state.replace('utah','UT')
df.state = df.state.replace('nevada','NV')
df.state = df.state.replace('new-hampshire', 'NH')
df.state = df.state.replace('oklahoma','OK')
df.state = df.state.replace('nebraska','NE')
df.state = df.state.replace('louisiana', 'LA')
df.state = df.state.replace('delaware','DE')
df.state = df.state.replace('iowa','IA')
df.state = df.state.replace('kentucky','KY')
df.state = df.state.replace('rhode-island','RI')
df.state = df.state.replace('connecticut','CT')
df.state = df.state.replace('south-dakota','SD')
df.state = df.state.replace('hawaii','HI')
df.state = df.state.replace('washington-dc','DC')
df.state = df.state.replace('wyoming','WY')
df.state = df.state.replace('arkansas','AK')
df.state = df.state.replace('alabama', 'AL')
df.state = df.state.replace('west-virginia','WV')
df.state = df.state.replace('north-dakota','ND')
df.state = df.state.replace('puerto-rico','PR')
df.state = df.state.replace('alaska','AK')

In [12]:
df.state.value_counts()

CA    284
CO    182
OR    156
WA    151
NY    107
PA    107
FL     99
MI     94
TX     90
WI     77
IL     74
MA     71
NC     61
OH     56
GA     47
ME     44
MD     43
VA     43
NJ     41
MN     40
NM     37
AZ     34
SC     32
MT     32
IN     31
VT     28
ID     28
TN     28
KS     26
MO     21
UT     21
AK     21
NV     18
NH     18
OK     16
LA     15
NE     15
DE     14
IA     13
RI     12
KY     12
SD     11
CT     11
HI     10
DC      9
WY      8
AL      6
WV      5
ND      5
PR      3
Name: state, dtype: int64

In [13]:
df.sample(25)

,brewery_name,type,address,website,state,state_breweries,address_2,city,state_2,zipcode,4
1033,Frederick Brewing Company,Microbrewery,"103 S. Carroll St., Frederick, Maryland, 21701",http://www.wildgoosebrewery.com/,MD,43,103 S. Carroll St.,Frederick,Maryland,21701,None
1118,Boston Beer Company,Microbrewery,"30 Germania St., Boston, Massachusetts, 2130",http://www.samadams.com/,MA,71,30 Germania St.,Boston,Massachusetts,2130,None
178,Elizabeth Street Brewery,Brewpub,"San Francisco, California, 94114",http://www.elizabethstreetbrewery.com/,CA,284,San Francisco,California,94114,None,None
1608,Boston Beer Co - DBA Samuel Adams Brewing Co.,RegionalBrewery,"1625 Central Parkway, Cincinnati, Ohio, 45214",http://www.samadams.com/,OH,56,1625 Central Parkway,Cincinnati,Ohio,45214,None
581,C.B. and Potts/Big Horn Brewery-Westminister,Brewpub,"1257 W 120th Ave, Westmister, Colorado, 80234",http://www.cbpotts.com/,CO,182,1257 W 120th Ave,Westmister,Colorado,80234,None
348,Iron Barley,Brewpub,"5510 Virginia, St. Louis, Missouri, 63111",http://www.ironbarley.com/,MO,21,5510 Virginia,St. Louis,Missouri,63111,None
1869,Dock Street Brewing Co. #3,ContractBrewery,"225 E. City Line Ave., Bala Cynwyd, Pennsylvan...",...,PA,107,225 E. City Line Ave.,Bala Cynwyd,Pennsylvania,19004,None
1666,Rock Creek Tavern,Brewpub,"10000 NW Old Cornelius Pass Rd., Hillsboro, Or...",http://www.mcmenamins.com/index.php?loc=48,OR,156,10000 NW Old Cornelius Pass Rd.,Hillsboro,Oregon,97124,None
168,Firestone Walker Brewing Co.,Brewpub,"620 McMurray Rd., Buellton, California, 93427",http://www.firestonewalker.com/,CA,284,620 McMurray Rd.,Buellton,California,93427,None
1984,Two Rows Restaurant & Brewery,Brewpub,"2400 University Blvd., Ste. 200, Houston, Texa...",http://www.tworows.com/,TX,90,2400 University Blvd.,Ste. 200,Houston,Texas,77005


In [14]:
df[4].unique()

array([None, ' 94117', ' 95814', ' 91730', ' 92131', ' 92507', ' 96161',
       ' 94520', ' 92054', ' 95662', ' 92593', ' 92647', ' 92780',
       ' 94518', ' 92701', ' 50315', ' 99669', ' 86336', ' 85281',
       ' 85603', ' 86001', ' 72201', ' 80301', ' 80020', ' 80265',
       ' 80234', ' 80227', ' 80021', ' Colorado, 80223', ' 33605',
       ' 33410', ' 33131', ' 33607', ' 33401', ' 32789', ' 30318',
       ' Hawaii, 96817', ' Hawaii, 96761', ' 96761', ' 60657', ' 60517',
       ' 61701', ' 61112', ' 60440', ' 4947', ' 4103', ' 4949', ' 20850',
       ' 2210', ' 48130', ' 49849', ' 3784', ' 3055', ' 3257', ' 87505',
       ' 13206', ' 10011', ' 28202', ' 28802', ' 73118', ' 97227',
       ' 97211', ' 97229', ' 97330', ' 97232', ' 97367', ' 97302',
       ' 19102', ' 37406', ' 77005', ' 75205', ' 78232', ' 84041',
       ' 98125', ' 98012', ' 99336', ' 98134', ' Washington, 98108',
       ' 20007', ' 20004', ' 20005', ' 20002'], dtype=object)

In [15]:
df['brewery_name'].nunique()

2308

Theirs 100 brewers on this list with the same name, i wonder if i get rid of the same brewerys with the same name if I will get rid of the address problem

In [26]:
name = []
zipcode = []
city =[]
state = []
zipcode = []
address = []

for data in df:  
    name.extend(df['brewery_name'].tolist())
    address.extend(df['address_2'].tolist())
    state.extend(df['state'].tolist())
    city.extend(df['city'].tolist())
    zipcode.extend(df['zipcode'].tolist())
    


In [27]:
api_key = 'Z2ljsgy2nJTRJMBNvMRouZIo4SCkaBm-9kD6QKJ6ZIyjnmfpawEegzJnIr786TzsAjYuuqqoNMzkQ47SR0AL94O0gAptOY4_fmLsbdudEUsngKeOKn4zxFNxuPFIX3Yx'
ENDPOINT = 'https://api.yelp.com/v3/businesses/matches'
HEADER = {'Authorization': 'bearer %s' % api_key}

In [29]:
data = []
for offest in range(0,2407 ,50):
        paramenters = {
            'name': name,
            'address1': address,
            'address2': "",
            'address3': "",
            "city" : city,
            'state': state,
            'country':"US",
            'zipcode': zipcode
        }
        response = requests.get(url = ENDPOINT, params = paramenters, headers = HEADER)
        if response.status_code == 200:
            data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 bad Reguest')
return data

400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest
400 bad Reguest


ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

SyntaxError: unexpected EOF while parsing (<ipython-input-25-dfd3494a0ffc>, line 4)